In [2]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [4]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
from fastapi import FastAPI, File, Form, UploadFile, Request
from typing import Optional
from fastapi.middleware.cors import CORSMiddleware
import torch
from pyngrok import ngrok, conf
import os
import cv2
import nest_asyncio
import uvicorn

In [5]:
device = torch.device("cuda")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", device_map='cuda', torch_dtype=torch.float16) 

model.to(device)

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [7]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# @app.get('/')
# async def root():
#     return {'hello': 'world'}

save_folder = "received_images"
os.makedirs(save_folder, exist_ok=True)
i = 0

@app.post("/")
async def upload_image(
    request: Request, 
    file: UploadFile = File(...), 
    prompt: Optional[str] = Form("")
):
    global i
    i += 1

    # Save uploaded image
    file_path = os.path.join(save_folder, f'received_image{i}.jpg')
    with open(file_path, "wb") as buffer:
        buffer.write(await file.read())

    # Load and process the image using OpenCV and PIL
    image = cv2.imread(file_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image_rgb)

    if prompt:
        prompt_text = f"Question: {prompt} Answer:"
    else:
        prompt_text = " "

    # Prepare inputs for the BLIP2 model
    inputs = processor(images=pil_image, text=prompt_text, return_tensors="pt").to(device, torch.float16)
    outputs = model.generate(**inputs)
    caption = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    return {
        "caption": caption,
    }

In [ ]:
conf.get_default().auth_token = ''

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://7ea4-34-116-67-248.ngrok-free.app                                               


INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
Expanding inputs for image tokens in BLIP-2 should be done in processing. Please follow instruction here (https://gist.github.com/zucchini-nlp/e9f20b054fa322f84ac9311d9ab67042) to update your BLIP-2 model. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Expanding inputs for image tokens in BLIP-2 should be done in processing. Please follow instruction here (https://gist.github.com/zucchini-nlp/e9f20b054fa322f84ac9311d9ab67042) to update your BLIP-2 model. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


INFO:     110.226.178.64:0 - "POST / HTTP/1.1" 200 OK
INFO:     110.226.178.64:0 - "GET / HTTP/1.1" 405 Method Not Allowed
INFO:     110.226.178.64:0 - "POST / HTTP/1.1" 200 OK
INFO:     110.226.178.64:0 - "POST / HTTP/1.1" 200 OK
INFO:     110.226.178.64:0 - "POST / HTTP/1.1" 200 OK
INFO:     110.226.178.64:0 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Finished server process [30]


KeyboardInterrupt: 